#### **Weather Data Pull**

In [62]:
import csv
import math
import time
import warnings
import operator
import statistics
import requests
import json
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

from dateutil.parser import parse
from datetime import datetime
from datetime import date, timedelta
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
from google.cloud import storage

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.6f}'.format

In [2]:
from pandas.io.json import json_normalize

def ws_historical_data(start, lat, long, period='day', fields='all'):
    '''
    If duration is more than 1 year separate calls should be used
    Timestamp should be converted to ISO 8601 format
    Docstring with examples and function return values:
    
    Input :
    start - (%Y-%m-%d) format
    end - (%Y-%m-%d) format
    lat - latitude 
    long - longitude
    period - hour, day (default=day)
    
    Output : return a collection of weather historical data for a latitude/longitude point
    
    '''
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

    proxyDict = { 
                "http"  : http_proxy, 
                "https" : https_proxy, 
                "ftp"   : ftp_proxy
                 }
    
    key = 'e721181f854ac2268ee8'
    start = pd.to_datetime(start,format='%Y-%m-%d')
    
    start = start.strftime('%Y-%m-%dT%H:%M:%S')
    
    weather_ = pd.DataFrame()
    link = 'https://api.weathersource.com/v1/'+key+'/points/'+lat+','+long+'/history.json?period='+period+'&timestamp_eq='+start+'&fields='+fields
    print(link)
    response = requests.get(link, headers=headers,proxies=proxyDict)
    json_obj = json.loads(response.content.decode('utf-8'))
    weather_ = json_normalize(json_obj)
    
    return weather_

In [46]:
df_ads = pd.read_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_V10.4.2020_09222020.csv')
print(df_ads.shape)
print('\n')
display(df_ads.head())

(13676, 100)




,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,CALL_QTY,DOWNSTREAM_CUST_QTY,KEY_CUST_QTY,ETR_DATETIME,CUST_QTY,DOWNSTREAM_KVA_VAL,KVA_VAL,DAY_FLAG,TTR,POLE_CLUE_FLG,PART_LIGHT_CLUE_FLG,EMERGENCY_CLUE_FLG,POWER_OUT_CLUE_FLG,OPEN_DEVICE_CLUE_FLG,TREE_CLUE_FLG,WIRE_DOWN_CLUE_FLG,IVR_CLUE_FLG,EQUIPMENT_CLUE_FLG,TRANSFORMER_CLUE_FLG,OH_CAUSE_FLG,UG_CAUSE_FLG,ANIMAL_CAUSE_FLG,WEATHER_CAUSE_FLG,WEATHER_COLD_CAUSE_FLG,PUBLIC_CAUSE_FLG,WEATHER_LIGHTNING_CAUSE_FLG,WEATHER__SNOW_CAUSE_FLG,WEATHER__WIND_CAUSE_FLG,WEATHER__HEAT_CAUSE_FLG,CUST_REQUEST_CAUSE_FLG,WEATHER__FLOOD_CAUSE_FLG,STREET_CAUSE_FLG,SUBSTATION_CAUSE_FLG,TREE_CAUSE_FLG,MISCELLANEOUS_CAUSE_FLG,NO_CAUSE_FLG,PLANNED_CAUSE_FLG,NO_OUTAGE_CAUSE_FLG,FUSE_OCCURN_FLG,CUST_EQUIP_OCCURN_FLG,POLE_OCCURN_FLG,TRANSFORMER_OCCURN_FLG,METER_OCCURN_FLG,SERVICE_OCCURN_FLG,CABLE_OCCURN_FLG,ST_OCCURN_FLG,FIRE_OCCURN_FLG,FOUND_OPEN_OCCURN_FLG,PUBLIC_SAFETY_OCCURN_FLG,WIRE_OCCURN_FLG,SWITCH_OCCURN_FLG,REGULATOR_OCCURN_FLG,CUTOUT_OCCURN_FLG,CAP_BANK_OCCURN_FLG,RECLOSER_OCCURN_FLG,OH_OCCURN_FLG,PRIORITY_VAL_1.0,PRIORITY_VAL_2.0,PRIORITY_VAL_3.0,PRIORITY_VAL_5.0,CITY_NAM,LAT,LONG,ZONE,NO_OF_POWER_OUT_CLUE_PER_DAY,NO_OF_OPEN_DEVICE_CLUE_PER_DAY,NO_OF_IVR_CLUE_PER_DAY,NO_OF_ANIMAL_CAUSE_PER_DAY,NO_OF_WIRE_OCCURN_PER_DAY,Marker_Location,Hour_Sin,Hour_Cos,RANK_SUBSEQUENT_OUTAGES,LIVE_OUTAGE,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Weekday,Weekend_flag,Live_outage_group,Priority_Customer_Qty,Dis_From_Live_Centriod_div_Cust_qty,Priority_Dist_Customer_Qty,Dispatch_Location,Date
0,2002645939,2001465208.00,462-B/97,2020-01-01 00:03:00,2020-01-01 02:30:00,3209.00,FUSE,320.00,26,30.00,1.00,2020-01-01 03:45:00,30.00,100.00,100.00,0,147.00,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.79,-86.19,ZONE4,37,2,17,8,5,Marker11,0.00,1.00,1.00,0,0,0,0,0,0,0,0,0,0,0,Wednesday,False,1,1.00,0.18,1.00,MILL,2020-01-01
1,2002645942,2001465216.00,481CB/195,2020-01-01 00:50:44,2020-01-01 01:15:40,2902.00,1TBOH,290.00,1,1.00,0.00,2020-01-01 04:30:00,1.00,0.00,0.00,0,24.93,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.80,-85.99,ZONE3,37,2,17,8,5,Marker13,0.00,1.00,2.00,1,1,1,1,1,1,1,1,1,1,1,Wednesday,False,1,2.00,5.39,2.00,ENGLISH,2020-01-01
2,2002646057,2001465296.00,281-B/193,2020-01-01 03:10:16,2020-01-01 11:46:19,3607.00,FUSE,360.00,3,2.00,0.00,2020-01-01 07:45:00,2.00,25.00,25.00,0,516.05,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,2,1.00,0.00,1.00,ARL.,2020-01-01
3,2002645992,2001465223.00,281-B/4,2020-01-01 03:10:16,2020-01-01 05:23:41,3607.00,SWITCH,360.00,3,10.00,1.00,2020-01-01 08:00:00,10.00,152.00,152.00,0,133.42,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.01,ZONE3,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,3,1.00,0.00,1.00,ARL.,2020-01-01
4,2002645993,2001465223.00,280-B/81,2020-01-01 03:10:16,2020-01-01 04:18:28,3607.00,SWITCH,360.00,70,205.00,0.00,2020-01-01 05:45:00,205.00,2480.70,2480.70,0,68.20,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,INDIANAPOLIS,39.88,-86.02,ZONE3,37,2,17,8,5,Marker5,0.71,0.71,3.00,0,0,0,1,2,2,2,2,2,2,2,Wednesday,False,4,2.00,0.02,1.00,ARL.,2020-01-01


In [47]:
df_ads = df_ads[['Marker_Location', 'Date']]
df_ads.drop_duplicates(subset=['Marker_Location', 'Date'], keep='first', inplace=True)


In [48]:
df_ads = df_ads[(df_ads.Marker_Location != 'Markernan')]
df_ads.reset_index(drop=True, inplace=True)
print(df_ads.shape)

(2502, 2)


In [49]:
df_ads['Marker_Location'] = df_ads['Marker_Location'].astype('str')
df_ads['Date'] = df_ads['Date'].astype('str')

In [50]:
def marker_location_mapping(marker_name):
    
    # latitude of the location markers
    sites_latitude = {
        'Marker1' : '39.9613','Marker2' : '39.8971','Marker3' : '39.9060',
        'Marker4' : '39.9024','Marker5' : '39.8960','Marker6' : '39.8339',
        'Marker7' : '39.8412','Marker8' : '39.8381','Marker9' : '39.8386',
        'Marker10' : '39.7579','Marker11' : '39.7621','Marker12' : '39.7621',
        'Marker13' : '39.7695','Marker14' : '39.6617','Marker15' : '39.6639',
        'Marker16' : '39.6702','Marker17' : '39.6744','Marker18' : '39.5909',
        'Marker19' : '39.5295','Marker20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker1' : '-86.4034','Marker2' : '-86.3045','Marker3' : '-86.2001',
        'Marker4' : '-86.0738','Marker5' : '-85.9783','Marker6' : '-86.3155',
        'Marker7' : '-86.2056','Marker8' : '-86.0985','Marker9' : '-85.9811',
        'Marker10' : '-86.3155','Marker11' : '-86.2042','Marker12' : '-86.0923',
        'Marker13' : '-85.9708','Marker14' : '-86.2935','Marker15' : '-86.1823',
        'Marker16' : '-86.0669','Marker17' : '-85.9557','Marker18' : '-86.4212',
        'Marker19' : '-86.5874','Marker20' : '-86.2743'
    }
    
    value1, value2 = sites_latitude.get(marker_name), sites_longitude.get(marker_name)
    
    return value1, value2

In [51]:
df_ads['LAT'], df_ads['LONG'] = zip(*df_ads.apply(lambda row: marker_location_mapping(row['Marker_Location']),axis=1))
df_ads

,Marker_Location,Date,LAT,LONG
0,Marker11,2020-01-01,39.7621,-86.2042
1,Marker13,2020-01-01,39.7695,-85.9708
2,Marker5,2020-01-01,39.8960,-85.9783
3,Marker4,2020-01-01,39.9024,-86.0738
4,Marker8,2020-01-01,39.8381,-86.0985
...,...,...,...,...
2497,Marker2,2020-07-10,39.8971,-86.3045
2498,Marker13,2020-07-10,39.7695,-85.9708
2499,Marker16,2020-07-10,39.6702,-86.0669
2500,Marker7,2020-07-10,39.8412,-86.2056


In [10]:
# latitude of the location markers
sites_latitude = {
    'Marker1' : '39.9613','Marker2' : '39.8971','Marker3' : '39.9060','Marker4' : '39.9024','Marker5' : '39.8960','Marker6' : '39.8339',
    'Marker7' : '39.8412','Marker8' : '39.8381','Marker9' : '39.8386','Marker10' : '39.7579','Marker11' : '39.7621','Marker12' : '39.7621',
    'Marker13' : '39.7695','Marker14' : '39.6617','Marker15' : '39.6639','Marker16' : '39.6702','Marker17' : '39.6744','Marker18' : '39.5909',
    'Marker19' : '39.5295','Marker20' : '39.5475'
    }
    
# longitude of the location markers
sites_longitude = {
    'Marker1' : '-86.4034','Marker2' : '-86.3045','Marker3' : '-86.2001','Marker4' : '-86.0738','Marker5' : '-85.9783','Marker6' : '-86.3155',
    'Marker7' : '-86.2056','Marker8' : '-86.0985','Marker9' : '-85.9811','Marker10' : '-86.3155','Marker11' : '-86.2042','Marker12' : '-86.0923',
    'Marker13' : '-85.9708','Marker14' : '-86.2935','Marker15' : '-86.1823','Marker16' : '-86.0669','Marker17' : '-85.9557','Marker18' : '-86.4212',
    'Marker19' : '-86.5874','Marker20' : '-86.2743'
    }


Marker_Location = ['Marker1', 'Marker2', 'Marker3', 'Marker4', 'Marker5', 'Marker6', 'Marker7', 'Marker8', 'Marker9', 'Marker10',
                   'Marker11', 'Marker12', 'Marker13', 'Marker14', 'Marker15', 'Marker16', 'Marker17', 'Marker18', 'Marker19', 'Marker20']

In [ ]:
waethersourcefiles_historical = []
ws_master = pd.DataFrame()

value1 = 0.0
value2 = 0.0

for j in range(len(df_ads)):
    time.sleep(2)
    value1 = sites_latitude.get(df_ads['Marker_Location'][j])
    value2 = sites_longitude.get(df_ads['Marker_Location'][j])
    
    waethersource_data_historical = ws_historical_data(start=df_ads.Date[j], lat=value1, long=value2)
    waethersourcefiles_historical.append(waethersource_data_historical)

waethersource_df_his = pd.concat(waethersourcefiles_historical)
waethersource_df_his.reset_index(drop=True, inplace=True)

https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.7621,-86.2042/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.7695,-85.9708/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.8960,-85.9783/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.9024,-86.0738/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.8381,-86.0985/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.7579,-86.3155/history.json?period=day&timestamp_eq=2020-01-01T00:00:00&fields=all
https://api.weathersource.com/v1/e721181f854ac2268ee8/points/39.6639,-86.1823/history.json?period=day&timestamp_eq=2020-01-0

In [52]:
waethersource_df_his.head()

,latitude,longitude,timestamp,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,presTendMin,presTendAvg,presTendMax,radSolarMin,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowDepth,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax
0,39.76,-86.20,2020-01-01 05:00:00+00:00,0,27,100,25.50,27.90,29.80,26.20,33.40,42.80,31.80,38.60,46.70,1007.00,1010.70,1013.50,0.00,-1.00,-0.20,0.70,0,118.60,499.20,2846.90,43.20,67.00,89.00,981.60,984.80,987.40,0.00,0.00,3.00,3.30,3.50,31.80,38.60,46.70,26.20,33.40,42.80,220,216,218,5.00,7.20,10.40,12.50,18.00,28.60,13.00,17.90,27.70,30.50,34.70,39.30
1,39.77,-85.97,2020-01-01 05:00:00+00:00,0,29,100,25.50,27.80,30.10,25.20,31.30,41.10,32.10,37.80,46.00,1007.20,1010.70,1013.50,0.00,-1.10,-0.20,0.60,0,117.50,502.50,2820.20,44.40,68.30,88.00,977.70,980.70,983.20,0.00,0.00,3.00,3.30,3.60,32.10,37.80,46.00,25.20,31.30,41.10,221,217,219,5.20,9.30,12.00,12.90,18.40,28.60,13.50,18.40,28.40,30.80,34.10,38.80
2,39.90,-85.98,2020-01-01 05:00:00+00:00,0,28,100,26.40,28.20,30.60,25.50,32.00,41.70,31.90,37.70,45.80,1007.00,1010.50,1013.30,0.00,-1.00,-0.20,0.60,0,117.90,501.10,2829.40,46.00,69.90,88.10,978.10,981.20,983.70,0.00,0.00,3.10,3.30,3.70,31.90,37.70,45.80,25.50,32.00,41.70,222,217,220,4.00,7.60,9.50,12.70,18.40,28.40,13.30,17.90,26.80,30.50,34.20,38.80
3,39.90,-86.07,2020-01-01 05:00:00+00:00,0,27,100,26.70,28.30,30.70,25.90,32.30,41.50,31.90,38.30,46.00,1006.90,1010.50,1013.20,0.00,-1.00,-0.20,0.60,0,117.40,497.40,2817.30,47.00,69.00,89.70,979.40,982.50,985.00,0.00,0.00,3.10,3.30,3.70,31.90,38.30,46.00,25.90,32.30,41.50,221,218,220,5.30,8.50,12.20,12.40,18.20,28.00,13.30,17.90,26.80,30.50,34.60,39.10
4,39.84,-86.10,2020-01-01 05:00:00+00:00,0,27,100,26.60,28.30,30.70,26.00,32.60,41.90,32.10,38.50,46.20,1007.00,1010.60,1013.30,0.00,-1.10,-0.20,0.70,0,117.70,498.70,2823.90,46.00,67.80,90.50,980.30,983.40,986.00,0.00,0.00,3.10,3.30,3.70,32.10,38.50,46.20,26.00,32.60,41.90,221,217,220,6.00,8.60,11.50,12.40,18.10,28.20,13.30,17.90,27.10,30.90,34.80,39.20


In [80]:
print(waethersource_df_his.shape)
print(df_ads.shape)

(2502, 58)
(2502, 4)


In [53]:
waethersource_df_his['timestamp'] = pd.to_datetime(waethersource_df_his['timestamp'], utc=True)

In [55]:
df_ads['Date'] = pd.to_datetime(df_ads['Date'], utc=True)

In [74]:
df_ads.reset_index(drop=True, inplace=True)
waethersource_df_his.reset_index(drop=True, inplace=True)

In [77]:
merged_data = pd.merge(df_ads, waethersource_df_his, left_index=True, right_index=True)

In [81]:
merged_data.head()

,Marker_Location,Date,LAT,LONG,latitude,longitude,timestamp,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,presTendMin,presTendAvg,presTendMax,radSolarMin,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowDepth,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax
0,Marker11,2020-01-01 00:00:00+00:00,39.7621,-86.2042,39.7621,-86.2042,2020-01-01 05:00:00+00:00,0,27,100,25.500000,27.900000,29.800000,26.200000,33.400000,42.800000,31.800000,38.600000,46.700000,1007.000000,1010.700000,1013.500000,0.000000,-1.000000,-0.200000,0.700000,0,118.600000,499.200000,2846.900000,43.200000,67.000000,89.000000,981.600000,984.800000,987.400000,0.000000,0.000000,3.000000,3.300000,3.500000,31.800000,38.600000,46.700000,26.200000,33.400000,42.800000,220,216,218,5.000000,7.200000,10.400000,12.500000,18.000000,28.600000,13.000000,17.900000,27.700000,30.500000,34.700000,39.300000
1,Marker13,2020-01-01 00:00:00+00:00,39.7695,-85.9708,39.7695,-85.9708,2020-01-01 05:00:00+00:00,0,29,100,25.500000,27.800000,30.100000,25.200000,31.300000,41.100000,32.100000,37.800000,46.000000,1007.200000,1010.700000,1013.500000,0.000000,-1.100000,-0.200000,0.600000,0,117.500000,502.500000,2820.200000,44.400000,68.300000,88.000000,977.700000,980.700000,983.200000,0.000000,0.000000,3.000000,3.300000,3.600000,32.100000,37.800000,46.000000,25.200000,31.300000,41.100000,221,217,219,5.200000,9.300000,12.000000,12.900000,18.400000,28.600000,13.500000,18.400000,28.400000,30.800000,34.100000,38.800000
2,Marker5,2020-01-01 00:00:00+00:00,39.896,-85.9783,39.896,-85.9783,2020-01-01 05:00:00+00:00,0,28,100,26.400000,28.200000,30.600000,25.500000,32.000000,41.700000,31.900000,37.700000,45.800000,1007.000000,1010.500000,1013.300000,0.000000,-1.000000,-0.200000,0.600000,0,117.900000,501.100000,2829.400000,46.000000,69.900000,88.100000,978.100000,981.200000,983.700000,0.000000,0.000000,3.100000,3.300000,3.700000,31.900000,37.700000,45.800000,25.500000,32.000000,41.700000,222,217,220,4.000000,7.600000,9.500000,12.700000,18.400000,28.400000,13.300000,17.900000,26.800000,30.500000,34.200000,38.800000
3,Marker4,2020-01-01 00:00:00+00:00,39.9024,-86.0738,39.9024,-86.0738,2020-01-01 05:00:00+00:00,0,27,100,26.700000,28.300000,30.700000,25.900000,32.300000,41.500000,31.900000,38.300000,46.000000,1006.900000,1010.500000,1013.200000,0.000000,-1.000000,-0.200000,0.600000,0,117.400000,497.400000,2817.300000,47.000000,69.000000,89.700000,979.400000,982.500000,985.000000,0.000000,0.000000,3.100000,3.300000,3.700000,31.900000,38.300000,46.000000,25.900000,32.300000,41.500000,221,218,220,5.300000,8.500000,12.200000,12.400000,18.200000,28.000000,13.300000,17.900000,26.800000,30.500000,34.600000,39.100000
4,Marker8,2020-01-01 00:00:00+00:00,39.8381,-86.0985,39.8381,-86.0985,2020-01-01 05:00:00+00:00,0,27,100,26.600000,28.300000,30.700000,26.000000,32.600000,41.900000,32.100000,38.500000,46.200000,1007.000000,1010.600000,1013.300000,0.000000,-1.100000,-0.200000,0.700000,0,117.700000,498.700000,2823.900000,46.000000,67.800000,90.500000,980.300000,983.400000,986.000000,0.000000,0.000000,3.100000,3.300000,3.700000,32.100000,38.500000,46.200000,26.000000,32.600000,41.900000,221,217,220,6.000000,8.600000,11.500000,12.400000,18.100000,28.200000,13.300000,17.900000,27.100000,30.900000,34.800000,39.200000


In [84]:
merged_data.to_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Historical_Data/Weather_Data_2020/IPL_Weather_Data2020.csv')